In [ ]:
library(aws.signature)
library(aws.s3)
library(tree)
library(colorspace)

use_credentials(profile = "handson2", file = "./credentials")
# Sys.getenv()

In [ ]:
rgb = read.csv('rgb.csv')
head(rgb)

In [ ]:
# 컬림 설명
# NAME: 이름
# COL: 헥사로 표현된 RGB 정보
# R: Red [0-255]
# G: Green [0-255]
# B: Blue [0-255]
# LABEL: 분류 된 색상 정보
# SOURCE: 해당 데이터의 출처 정보
# - REFERENCE: https://en.wikipedia.org/wiki/List_of_RAL_colors
# - PRODUCT: 상품이미지에서 추출한 대표 색상정보
str(rgb)

In [ ]:
plot(table(rgb$LABEL))

In [ ]:
dat = rgb[, c("R", "G", "B", "LABEL")]
col.type = "rgb"

col.type = "lab"
# lab = as(RGB(dat[, "R"]/255, dat[, "G"]/255, dat[, "B"]/255), "LAB")
# dat = as.data.frame(coords(lab))
# dat$LABEL = rgb$LABEL

set.seed(0908)
index = sample(round(nrow(dat) * 0.7))

# 훈련 데이터와 평가 데이터를 분리
train.dat = dat[index, ]
test.dat = dat[-index, ]

write.csv(train.dat, file = "train.csv")
write.csv(test.dat, file = "test.csv")

In [ ]:
bucket.name = "sagemaker-handson-2-<userid>" # <userid> 대신 자신의 아이디를 넣어주세요
result = try(bucket_exists(bucket.name),
                  silent = T)
if(class(result) == "try-error") {
  put_bucket(bucket.name)  
}

# 원본 데이터, 학습 데이터, 평가 데이터를 S3에 업로드
put_object(file = "rgb.csv", bucket = bucket.name)
put_object(file = "train.csv", bucket = file.path(bucket.name, "data", col.type, "train"))
put_object(file = "test.csv", bucket = file.path(bucket.name, "data", col.type, "test"))

In [ ]:
# tree 모델을 학습 데이터로 학습
model = tree(LABEL ~ ., data = train.dat)
summary(model)

In [ ]:
# 시각화
plot(model); text(model)

In [ ]:
# 평가 데이터를 이용해서 모델 검증하기
result = predict(model, test.dat)
# colnames(result)[apply(result, 1, which.max)]
# table(test.dat$LABEL == colnames(result)[apply(result, 1, which.max)])

# Confusion Matrix
table(test.dat$LABEL, colnames(result)[apply(result, 1, which.max)])

In [ ]:
save(model, file = "model.Rdata")
system("tar -cvzf model.tar.gz model.Rdata")

# 학습 한 모델을 S3에 업로드
# 향후에 새로운 데이터를 입력하여 예측을 할 때 사용
put_object(file = "model.tar.gz", bucket = file.path(bucket.name, "model"))